In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import numpy as np
from bnb.problem import OptimizationProblem
from matplotlib import pyplot as plt
import matplotlib
from scipy.optimize import minimize
import pandas as pd
from bnb.fml_solver import FMLSolver
import warnings
from cvxopt.solvers import cp
import time
from numpy.linalg import matrix_rank
import seaborn as sns
from cvxopt import solvers, matrix, spdiag, log, mul
from cvxopt import matrix_repr, matrix_str
import pandas as pd
from scipy.special import xlogy
np.set_printoptions(precision=3)

sns.set()
matplotlib.rcParams.update({'font.size': 11, 'font.family': 'serif'})

In [106]:
# np.random.seed(2)
n, m = 20, 4

a0 = np.random.uniform(0, 2, size=(m, n))
b0 = np.random.uniform(0.1, 1.0, size=n)
w0 = np.random.uniform(size=m)
w0 /= np.sum(w0)

problem = OptimizationProblem(a0, b0, w0)
problem.k = problem.S * problem.w.reshape(1, -1) / problem.b.reshape(-1, 1)
x_lb, x_ub = problem.x_lb, problem.x_ub

solver = FMLSolver(problem)
# solver.solve()

k = solver.k
z_lb = np.exp(-problem.p_ub * problem.b)
z_ub = np.exp(-problem.p_lb * problem.b)

k_mat = matrix(problem.k)
S = np.exp(problem.A)

In [107]:
theta_start = matrix(np.hstack((x_lb, (z_lb + z_ub) / 2)))
r_start = matrix(np.ones(m + 1))

In [108]:
def gradient(theta):
    
    # Df (m + 1, n)
    x, z = theta[:m], theta[m:]
    x_, z_ = np.asarray(x)[:, 0], np.asarray(z)[:, 0]
    
    obj_grad = np.vstack((
        k_mat.T * mul(z, log(z)),  # m
        mul(1 + log(z), k_mat * x)  # n
    ))
    
    constr_grads = []
    for c in range(m):
        constr_grad = np.zeros(m + n)
        constr_grad[c] = 1 + S[c] @ z_
        constr_grad[m:] = x_[c] * S[c]
        constr_grads.append(constr_grad)
    constr_grads = np.vstack(constr_grads).T
    return matrix(np.hstack((obj_grad, constr_grads)).T)

np.asarray(gradient(theta_start)).shape

(5, 24)

In [109]:
def _get_obj_hess(x, z):
    # dfdf / dz_j dx_c = k_{jc} * (1 + log(z_j)) # m * n
    hess = np.zeros((n + m, n + m))
    dxdz = np.diag(1 + np.log(z)) @ k  # n * m
    hess[m:, :m] = dxdz
    hess[:m, m:] = dxdz.T
    diag = np.hstack((np.zeros(m), 1 / z * (k @ x)))
    np.fill_diagonal(hess, diag)
    return hess


def hess(theta, r):
    x_, z_ = theta[:m], theta[m:]
    x, z, = np.asarray(x_)[:, 0], np.asarray(z_)[:, 0]
    
    # hess objective + hess constraint 1 + hess constraint 2
    hess_ = r[0] * _get_obj_hess(x, z)
    
    for c in range(m):
        hess_[c, m:] += r[c + 1] * S[c]
        hess_[m:, c] += r[c + 1] * S[c]
        
    hess_ *= np.tri(*hess_.shape)
    
    return matrix(hess_)
    
# bounds = list(zip(x_lb, x_ub)) + list(zip(z_lb, z_ub))
np.asarray(hess(theta_start, r_start))

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   , -0.   , -0.   ,  0.   , -0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 1.112,  1.921,  2.246,  5.495,  0.759,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0. 

In [110]:
G = matrix(np.vstack((np.identity(m + n), - np.identity(m + n))))

h = matrix(np.hstack((
    x_ub,
    z_ub,
    - x_lb,
    - z_lb
)))


def _cnstr(theta, c):
    x, z = theta[:m, 0], theta[m:, 0]
    return x[c] + x[c] * (S[c] @ z) - 1


def _objective(theta):
    x, z = theta[:m], theta[m:]
    return np.sum(xlogy(z, z) * (problem.k @ x))


def F(x=None, z=None):
    
    # print(z)
    
    if x is None:
        return m, matrix(theta_start)
    
    if min(x) <= 0.0:
        return None

    x_ = np.asarray(x)
    f = matrix([_objective(x_)] + [_cnstr(x_, c) for c in range(m)])
    Df = gradient(x)    
    
    if z is None:
        return f, Df
    
    H = hess(x, z)
    # print(matrix_rank(np.vstack((H, Df, G))))

    return f, Df, H

f, Df, H = F(x=theta_start, z=r_start)
_cnstr(np.asarray(theta_start), 0), _objective(np.asarray(theta_start))

(-0.458436844583762, -2.116602448764481)

In [111]:
solvers.options["show_progress"] = False
solvers.options['maxiters'] = 100

In [116]:
sol = solvers.cp(F, G=G, h=h, kktsolver="ldl")
np.asarray(sol["x"])

array([[4.585e-01],
       [4.169e-01],
       [3.948e-01],
       [4.179e-01],
       [1.525e-04],
       [2.274e-04],
       [8.126e-04],
       [2.875e-02],
       [2.650e-04],
       [1.908e-02],
       [3.771e-04],
       [1.210e-01],
       [1.132e-01],
       [5.369e-04],
       [7.800e-02],
       [7.321e-03],
       [1.379e-02],
       [2.306e-02],
       [1.215e-02],
       [5.736e-03],
       [4.507e-04],
       [1.962e-03],
       [3.071e-04],
       [3.763e-04]])

In [ ]:
# Rank([H(x); A; Df(x); G]) < n


In [20]:
from scipy.special import xlogy
np.random.seed(1)
n, m = 10, 4

a0 = np.random.uniform(0, 4, size=(m, n))
b0 = np.random.uniform(0.001, 0.01, size=n)
w0 = np.random.uniform(size=m)
w0 /= np.sum(w0)

problem = OptimizationProblem(a0, b0, w0)
problem.k = problem.S * problem.w.reshape(1, -1) / problem.b.reshape(-1, 1)
x_lb, x_ub = problem.x_lb, problem.x_ub

solver = FMLSolver(problem)
solver.solve()

k = solver.k
z_lb = np.exp(-problem.p_ub * problem.b)
z_ub = np.exp(-problem.p_lb * problem.b)
min_z = np.min(z_lb)

z_lb /= min_z
z_ub /= min_z

# x_lb = np.asarray([0.19674673, 0.10397393, 0.01786247])
# x_ub = np.asarray([0.11, 0.11, 0.11])
# x_ub = x_lb + 0.0000001

S = np.exp(problem.A)

def cnstr(theta, c):
    x, z = theta[:m], theta[m:] * min_z
    return 1 - x[c] - x[c] * (S[c] @ z)  # >= 0

def objective(theta):
    x, z = theta[:m], theta[m:] * min_z
    return np.sum(xlogy(z, z) * (problem.k @ x))

def jac(theta):
    x, z = theta[:m], theta[m:] * min_z
    return np.hstack((
        xlogy(z, z) @ k,
        min_z * (1 + np.log(z)) * (k @ x)
    ))

constraints = (
    [{"type": "ineq", "fun": lambda theta, c=c: cnstr(theta, c)} for c in range(m)]
)

# theta_start = np.hstack(((x_lb + x_ub) / 2, (z_ub + z_lb) / 2))
theta_start = np.hstack((x_lb, z_lb))

bounds = list(zip(x_lb, x_ub)) + list(zip(z_lb, z_ub))

for cnstr_ in constraints:
    print('cnstr: ', cnstr_["fun"](theta_start))

opt = minimize(
    objective,
    theta_start,
    bounds=bounds,
    constraints=constraints,
    jac=jac,
    method="SLSQP",
    options={"maxiter": 1e6}
)

print(opt)

LB: -inf, UB: inf.
number of cubes:  16
LB: 666.2137515167631, UB: inf.
number of cubes:  64
LB: 669.5753889088821, UB: inf.
number of cubes:  800
LB: 695.1234016113383, UB: inf.
number of cubes:  5216
LB: 697.0491550668996, UB: inf.
number of cubes:  23040
LB: 699.2246791242945, UB: inf.
number of cubes:  56240
LB: 699.4133776859809, UB: 1588.7600576855075.
number of cubes:  68960
LB: 699.4630148257604, UB: 735.6768123202612.
number of cubes:  2352
LB: 699.4977137013334, UB: 699.5536744999304.
cnstr:  0.9289973502579139
cnstr:  0.965132941465093
cnstr:  0.9543953998266457
cnstr:  0.9510671411367578
     fun: -688.3079091030831
     jac: array([-1.49284468e+01, -5.62872197e+02, -9.55538660e+02, -9.53040972e+02,
       -6.28257727e-03, -2.07664618e-02, -1.05263711e-02, -1.79244653e-02,
       -9.72319218e-03, -1.20327298e-02, -8.73258269e-03, -1.10358940e-02,
       -2.44876761e-03, -1.14910086e-02])
 message: 'Positive directional derivative for linesearch'
    nfev: 46
     nit: 48
  

In [14]:
solver.cubes[0].center

array([0.19674673, 0.10397393, 0.01786247])

In [9]:
z_lb

array([5.47177689e-02, 2.90326675e-03, 2.19805771e-05, 1.04747199e-03,
       2.47209592e-04, 7.58726658e-03, 2.59601296e-04, 6.74620868e-05,
       1.13355631e-01, 1.45495366e-04])

In [25]:
z_lb = np.exp(-problem.p_ub * problem.b)
z_ub = np.exp(-problem.p_lb * problem.b)
S = np.exp(problem.A)

def cnstr_1(theta, c):
    z = theta[m:]
    return 1 / problem.x_lb[c] - 1 - S[c] @ z  # >= 0

def cnstr_2(theta, c):
    z = theta[m:]
    return 1 + S[c] @ z - 1 / problem.x_ub[c]   # >= 0

constraints = (
    [{"type": "ineq", "fun": lambda theta, c=c: cnstr_1(theta, c)} for c in range(m)] +
    [{"type": "ineq", "fun": lambda theta, c=c: cnstr_2(theta, c)} for c in range(m)]
)

theta_start = np.hstack(((problem.x_ub + problem.x_lb) / 2, (z_ub + z_lb) / 2))

bounds = list(zip(problem.x_lb, problem.x_ub)) + list(zip(z_lb, z_ub))

def objective(theta):
    x, z = theta[:m], theta[m:]
    return np.sum(z * np.log(z) * (problem.k @ x))

opt = minimize(
    objective,
    theta_start,
    bounds=bounds,
    constraints=constraints
)

opt

     fun: -3694.4727843837227
     jac: array([-2.50951657e+03, -2.68474533e+03, -2.37769562e+03,  6.10351562e-05,
        6.10351562e-05,  0.00000000e+00,  6.10351562e-05,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.5226932 , 0.59345371, 0.33204144, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944])

In [6]:
# from scipy.optimize import LinearConstraint

# constr_1 = LinearConstraint(S, -np.inf, 1 / problem.x_lb - 1)
# constr_2 = LinearConstraint(-S, -np.inf,  1 - 1 / problem.x_ub)

# constraints = [constr_1, constr_2]



TypeError: Constraints must be defined using a dictionary.

In [59]:
cnstr_2(theta_start)

array([ -8.80413801, -16.39376441, -35.66130005])